Run the following line if you don't have any of the libraries

In [1]:
!pip install statsmodels pandas numpy --upgrade --user

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [11]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.ar_model import AutoReg, ar_select_order
from statsmodels.tsa.api import acf, pacf, graphics, VAR
import data_prep_modified as dpm
import numpy as np

In [3]:
data = pd.read_csv("./data/output.csv")
list(data.columns) 

['Date',
 'Open_Pharm',
 'High_Pharm',
 'Low_Pharm',
 'Close_Pharm',
 'Adj Close_Pharm',
 'Volume_Pharm',
 'Open_Semicon',
 'High_Semicon',
 'Low_Semicon',
 'Close_Semicon',
 'Adj Close_Semicon',
 'Volume_Semicon',
 'Open_SNP500',
 'High_SNP500',
 'Low_SNP500',
 'Close_SNP500',
 'Adj Close_SNP500',
 'Volume_SNP500',
 'Open_Industrial',
 'High_Industrial',
 'Low_Industrial',
 'Close_Industrial',
 'Adj Close_Industrial',
 'Volume_Industrial',
 'Open_Energy',
 'High_Energy',
 'Low_Energy',
 'Close_Energy',
 'Adj Close_Energy',
 'Volume_Energy',
 'Open_Financial',
 'High_Financial',
 'Low_Financial',
 'Close_Financial',
 'Adj Close_Financial',
 'Volume_Financial',
 'Open_Tech',
 'High_Tech',
 'Low_Tech',
 'Close_Tech',
 'Adj Close_Tech',
 'Volume_Tech',
 'Open_Utilities',
 'High_Utilities',
 'Low_Utilities',
 'Close_Utilities',
 'Adj Close_Utilities',
 'Volume_Utilities',
 'Open_Consumer',
 'High_Consumer',
 'Low_Consumer',
 'Close_Consumer',
 'Adj Close_Consumer',
 'Volume_Consumer']

In [4]:
#variable prep commands, replace snp500 with variable of choice
snp500delta = dpm.get_perc_return(data, column_name="Adj Close_SNP500")
snp500delta.set_index('Date', inplace=True)
#for some reason, I cant put the set_index command in the function because of some error

In [5]:
snp500delta.head()

,%_returns
Date,
5/1/2010,0.003116
6/1/2010,0.000546
7/1/2010,0.004001
8/1/2010,0.002882
11/1/2010,0.001747


In [9]:
ar_1 = AutoReg(snp500delta, lags =1).fit()
ar_1.summary()
ar_1.aic

C:\Users\Hamu\AppData\Roaming\Python\Python37\site-packages\statsmodels\tsa\base\tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\Users\Hamu\AppData\Roaming\Python\Python37\site-packages\statsmodels\tsa\base\tsa_model.py:222: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  ' forecasting.', ValueWarning)


-9.143433697491265

In [10]:
pharmdelta = dpm.get_perc_return(data, column_name="Adj Close_Pharm")
pharmdelta.set_index('Date', inplace=True)

In [23]:
pharmdelta.columns = ['pharm_returns']

In [25]:
adl_df = pd.concat([snp500delta,pharmdelta], axis = 1)
adl_df

,%_returns,pharm_returns
Date,,
5/1/2010,0.003116,-0.009392
6/1/2010,0.000546,0.004966
7/1/2010,0.004001,-0.001647
8/1/2010,0.002882,0.000300
11/1/2010,0.001747,0.006899
...,...,...
16/3/2020,-0.119841,-0.089813
17/3/2020,0.059955,0.060510
18/3/2020,-0.051831,-0.052955


In [35]:
test = VAR(adl_df, exog = adl_df['pharm_returns']).fit(maxlags=15, ic='aic')
test.summary()

C:\Users\Hamu\AppData\Roaming\Python\Python37\site-packages\statsmodels\tsa\base\tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\Users\Hamu\AppData\Roaming\Python\Python37\site-packages\statsmodels\tsa\base\tsa_model.py:222: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  ' forecasting.', ValueWarning)


  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Mon, 06, Apr, 2020
Time:                     19:16:33
--------------------------------------------------------------------
No. of Equations:         2.00000    BIC:                   -90.0695
Nobs:                     2570.00    HQIC:                  -90.0753
Log likelihood:           108462.    FPE:                7.57476e-40
AIC:                     -90.0786    Det(Omega_mle):     7.56298e-40
--------------------------------------------------------------------
Results for equation %_returns
                   coefficient       std. error           t-stat            prob
--------------------------------------------------------------------------------
const                 0.000083         0.000120            0.695           0.487
pharm_returns         0.833727         0.011717           71.157           0.000

Results for equation pharm_returns
                   c

AttributeError: 'VARResults' object has no attribute 'endog_names'